In [21]:
# Import necessary libraries
from flask import Flask, request, jsonify
import yfinance as yf
from datetime import datetime
import pandas as pd
import numpy as np
from threading import Thread
import requests
import json

In [22]:
# Reset Flask app
try:
    app = Flask(__name__)
except:
    pass

In [23]:
# Initialize Flask app
app = Flask(__name__)

# Root URL
@app.route('/')
def home():
    return jsonify({'message': 'Welcome to the Flask API!'}), 200

# Favicon handler
@app.route('/favicon.ico')
def favicon():
    return '', 204

In [24]:
@app.route('/company_info', methods=['GET'])
def get_company_info():
    symbol = request.args.get('symbol')
    if not symbol:
        return jsonify({'error': 'Company symbol is required'}), 400
    try:
        company = yf.Ticker(symbol)
        info = company.info
        company_data = {
            'full_name': info.get('longName', 'N/A'),
            'business_summary': info.get('longBusinessSummary', 'N/A'),
            'industry': info.get('industry', 'N/A'),
            'sector': info.get('sector', 'N/A'),
            'key_officers': info.get('companyOfficers', 'N/A')
        }
        return jsonify(company_data)
    except Exception as e:
        return jsonify({'error': f'Failed to fetch company info: {str(e)}'})

In [25]:
@app.route('/stock_market_data', methods=['GET'])
def get_stock_market_data():
    symbol = request.args.get('symbol')
    if not symbol:
        return jsonify({'error': 'Company symbol is required'}), 400
    try:
        stock = yf.Ticker(symbol)
        info = stock.info
        stock_data = {
            'market_state': info.get('marketState', 'N/A'),
            'current_price': info.get('regularMarketPrice', info.get('currentPrice', 'N/A')),
            'price_change': info.get('regularMarketChange', 'N/A'),
            'percent_change': info.get('regularMarketChangePercent', 'N/A'),
            'volume': info.get('regularMarketVolume', 'N/A'),
            'market_cap': info.get('marketCap', 'N/A')
        }
        return jsonify(stock_data)
    except Exception as e:
        return jsonify({'error': f'Failed to fetch stock market data: {str(e)}'})

In [26]:
# Historical Market Data Endpoint
@app.route('/historical_market_data', methods=['POST'])
def get_historical_market_data():
    data = {
        'symbol': 'AAPL',
        'start_date': '2023-01-01',
        'end_date': '2023-12-31'
    }
    try:
        stock = yf.Ticker(data['symbol'])
        history = stock.history(start=data['start_date'], end=data['end_date'])
        history_data = history.reset_index().to_dict(orient='records')
        return jsonify(history_data)
    except Exception as e:
        return jsonify({'error': f'Failed to fetch historical data: {str(e)}'})

In [27]:
# Analytical Insights Endpoint
@app.route('/analytical_insights', methods=['POST'])
def get_analytical_insights():
    data = {
        'symbol': 'AAPL',
        'start_date': '2023-01-01',
        'end_date': '2023-12-31'
    }
    try:
        stock = yf.Ticker(data['symbol'])
        history = stock.history(start=data['start_date'], end=data['end_date'])
        avg_price = history['Close'].mean()
        max_price = history['Close'].max()
        min_price = history['Close'].min()
        volatility = history['Close'].std()
        sma_20 = history['Close'].rolling(window=20).mean().iloc[-1] if len(history) >= 20 else None
        insights = {
            'average_price': round(avg_price, 2),
            'highest_price': round(max_price, 2),
            'lowest_price': round(min_price, 2),
            'volatility': round(volatility, 2),
            '20_day_sma': round(sma_20, 2) if sma_20 else 'N/A',
            'trend': 'Uptrend' if sma_20 and history['Close'].iloc[-1] > sma_20 else 'Downtrend' if sma_20 else 'N/A'
        }
        return jsonify(insights)
    except Exception as e:
        return jsonify({'error': f'Failed to fetch analytical insights: {str(e)}'})

In [28]:
def run_flask():
    app.run(host='0.0.0.0', port=5000, debug=False, use_reloader=False)

# Start Flask in a separate thread
flask_thread = Thread(target=run_flask)
flask_thread.daemon = True
flask_thread.start()
print("Flask server is running at http://127.0.0.1:5000")

Flask server is running at http://127.0.0.1:5000
 * Serving Flask app '__main__'


 * Debug mode: off


 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://192.168.1.112:5000
Press CTRL+C to quit


In [29]:
# Wait for server to start
import time
time.sleep(2)

# Test endpoints
print("Testing endpoints...")
base_url = "http://127.0.0.1:5000"

# Test root endpoint
response = requests.get(f"{base_url}/")
print("\nRoot endpoint:", response.json())

# Test company info with symbol
response = requests.get(f"{base_url}/company_info?symbol=AAPL")
print("\nCompany info:", json.dumps(response.json(), indent=2))

# Test stock market data with symbol
response = requests.get(f"{base_url}/stock_market_data?symbol=AAPL")
print("\nStock market data:", json.dumps(response.json(), indent=2))

# Test historical market data
historical_data = {
    "symbol": "AAPL",
    "start_date": "2023-01-01",
    "end_date": "2023-12-31"
}
response = requests.post(f"{base_url}/historical_market_data", json=historical_data)
print("\nHistorical data (first 2 records):", json.dumps(response.json()[:2], indent=2))

# Test analytical insights
response = requests.post(f"{base_url}/analytical_insights", json=historical_data)
print("\nAnalytical insights:", json.dumps(response.json(), indent=2))

Testing endpoints...

Root endpoint: {'message': 'Welcome to the Flask API!'}

Company info: {
  "business_summary": "Apple Inc. designs, manufactures, and markets smartphones, personal computers, tablets, wearables, and accessories worldwide. The company offers iPhone, a line of smartphones; Mac, a line of personal computers; iPad, a line of multi-purpose tablets; and wearables, home, and accessories comprising AirPods, Apple TV, Apple Watch, Beats products, and HomePod. It also provides AppleCare support and cloud services; and operates various platforms, including the App Store that allow customers to discover and download applications and digital content, such as books, music, video, games, and podcasts, as well as advertising services include third-party licensing arrangements and its own advertising platforms. In addition, the company offers various subscription-based services, such as Apple Arcade, a game subscription service; Apple Fitness+, a personalized fitness service; Appl